In [ ]:
driver.close()

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import datetime as dt
import pandas as pd
from collections import Counter
import os
# os.chdir('/content/drive/MyDrive/015GithubRepos/Class_DatabaseSystem')

In [ ]:
# data_PATH = '/content/drive/MyDrive/015GithubRepos/Class_DatabaseSystem'

In [ ]:
driver = webdriver.Chrome('J:/내 드라이브/015GithubRepos/Class_DatabaseSystem/chromedriver.exe')
data_PATH = 'J:/내 드라이브/015GithubRepos/Class_DatabaseSystem/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
urls = [
    'https://search.musinsa.com/category/001001', # 반팔 티셔츠
    'https://search.musinsa.com/category/003002', # 데님팬츠
    'https://search.musinsa.com/category/003008', # 슈트 팬츠/슬랙스
    'https://search.musinsa.com/category/001002', # 셔츠/블라우스
    'https://search.musinsa.com/category/003004', # 트레이닝/조거 팬츠
    'https://search.musinsa.com/category/003009' # 숏 팬츠
#     'https://search.musinsa.com/category/005004', # 샌들
]

In [ ]:
# 무신사 홈페이지로 이동한 다음 우측 하단 '전체', '남성', '여성' 중 '여성'을
# 클릭해야함
# 홈페이지로 들어가고 끝나면, 무신사와 우신사 url이 같아서 무신사로 들어가게됨
driver.get('https://store.musinsa.com/app/')
driver.find_element_by_css_selector('button.global-filter__button.global-filter__button--wusinsa').click()

In [ ]:
# 수집할 데이터 리스트들 ##############################
product_names = []
categories = []

retypes = []

# purchase size, sex, height, weight
meta_pshw = []
meta_sizes = []
meta_brights = []
meta_colors = []
meta_thicks = []

rvs = []
scores = []
helpfuls = []
uploadAts = []

collectAts = []
############################## 수집할 데이터 리스트들 #

# pattern for max page
p_maxp = re.compile(r'[\d,]+')

n = 10
nn = 3200


---

# 뜯어서 검증

In [ ]:
cateurl

'https://search.musinsa.com/category/001001'

In [ ]:
# ii : 카테고리별 수집 개수 cnt
# i : product별 수집 개수 cnt
ii = 0

# 카테고리로 이동(ex. 반팔티셔)
driver.get(cateurl)
time.sleep(3)

dom = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(0.5)

#  카테고리(반팔티셔츠)별 제품들의 url들 구하기

driver.get(cateurl)
dom = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(0.5)
cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
product_urls = [_['href'] for _ in cate_products]

for _ in range(4, 13):
    driver.find_element_by_css_selector(f'div.sorter-box.box div.pagination div.wrapper a:nth-of-type({_})').click()
    time.sleep(0.5)
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    page_cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
    page_product_urls = [_['href'] for _ in page_cate_products]
    product_urls += page_product_urls
#     print(f'_ : {_}, page : {_-2}')


# product 개수 세기
pn = 0

In [ ]:
for product_url in product_urls:
    break

In [ ]:
product_url

'https://store.musinsa.com/app/goods/324416'

In [ ]:
product_url = 'https://store.musinsa.com/app/goods/558627'

In [ ]:
pn += 1
# product로 이동
driver.get(product_url)
time.sleep(3)

dom = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(0.5)

product_name = dom.select_one('span.product_title em').text.strip()
category = dom.select_one('p.item_categories a:nth-of-type(2)').text.strip()


numofrv = dom.select_one('h3.title-box.font-mss span#review_total').text.strip()
prdid = product_url.split('/')[-1]
print(f'상품아이디: {prdid}, 본 상품 총 후기 개수 : {numofrv}')
# i == n이면 빠져나오기 위함
# product별 최대 100개씩
i = 0


# review types 1 : 스타일 후기, 2 : 상품사진후기, 3 : 일반 후기
# 체험단후기가 있는 경우, 2 : 스타일 후기, 3 : 상품사진후기, 4 : 일반 후기
if len(dom.select('div#estimateBox div.tabBox > ul.snb > li')) == 4:
    rvtypes = {2 : '스타일 후기', 3: '상품 사진 후기', 4: '일반 후기'}
else:
    rvtypes = {1 : '스타일 후기', 2: '상품 사진 후기', 3: '일반 후기'}

상품아이디: 558627, 본 상품 총 후기 개수 : 43


In [ ]:
len(dom)

1

In [ ]:
rvtype = 3

In [ ]:

# review type 화면으로 이동
# driver.find_element_by_css_selector(f'div#estimateBox div.tabBox > ul.snb > li:nth-of-type({rvtype})').click()
# time.sleep(1)

# # dom 불러오기
# dom = BeautifulSoup(driver.page_source, 'html.parser')
# time.sleep(0.5)

# page description : 최대 몇 페이지까지 있는지 텍스트 ('000페이지 중 000페이지')
# maxp : 해당 상품의 마지막 페이지 숫자
if dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg'):
    pagedes = dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg').text.strip()
    maxp = int(re.search(p_maxp, pagedes).group(0).replace(',', ''))

    # curr_num : 현재 페이지
    curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
    curr_num = int(curr_num)
else:  # 후기가 없는 경우
    maxp = 0


print(f'{rvtypes[rvtype]} max page : {maxp}')



# 일반 후기까지 다 했거나, i==100이면 리뷰타입 볼 필요 없이 다음 product로 넘어간다
if i == n or ii >= nn:
    print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
    

####################################### review type별 수집 #

일반 후기 max page : 2


In [ ]:
maxp

2

In [ ]:
rvtype = ['style_estimate_list', 'photo_estimate_list', 'estimate_list']

In [ ]:
for rvbox in dom.select(f'div#estimateBox div.tabBox div#{rvtype[2]} div.review-list'):
    print(rvbox.select_one('div.review-contents__text').text.strip())
    print()

S인데도 허리랑 허벅지쪽이 커요. 그래도 길이는 짧지 않아서 좋고, 만약 핏이 딱 떨어지는 숏팬츠를 원한다면 추천하지 않아요.

무난하게 입기 좋습니다. 다만 원단이 좀 얇아서 말려올라가긴 했네요.

길이가 막 길지도 짧지도 않고 적당해서 좋아요 또 예뻐요

색감 진짜이뻐요ㅠㅠ̑̈ 길이도 적당하고 핏도 딱 떨어지는게 굿굿 이번여름 자주입었어용ㅎㅅㅎ

평소에 입는 사이즈보다 약간 큰 편이며 적당한 하이웨이스트 스타일임

완전 아주 밝은 연청색입니다. 얇아서 여름에 시원하게 입을 수 있을것 같아요

편하고 핏도 이쁘구 시원한 소재구
완벽해요 ㅠㅠ♡ 올해 바지 쇼핑 성공!
이번에 첨 안 브랜드인데 더 사고싶네요

계속 입다보면 너무 a라인 같긴한데 그래두 예뻐용

저렴한 가격 대비
원단 도톰하고 색감 너무 시원해보여요

주름지는거는 재질이 얇아서 그런거 같아요 개인적으로 허리도 컸어요 그거 빼고는 괜찮았고요



In [ ]:
rvbox

In [ ]:
rvbox.select_one('span.review-list__rating__active').get('style')[-3] in ['2', '4', '6', '8']

False

In [ ]:
# paging ##################################
while maxp:

    # dom 새로 불러오기
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)

    # 현재 페이지 구하기
    curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
    curr_num = int(curr_num)
    print(f'current page number : {curr_num}, length of rvs : {len(rvs)}')

    # list of pages : 페이지 리스트
    lp = [int(_.text.strip()) if _.text.strip() != '' else 0 for _ in dom.select('div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a')]
    
    # 한 페이지에서 데이터 수집 ################################
    for rvbox in dom.select('div#estimateBox div.tabBox div#style_estimate_list div.review-list'):

        if rvbox.select_one('span.review-list__rating__active').get('style')[-3] in ['2', '4', '6', '8']:

            product_names.append(product_name)
            categories.append(category)

            retypes.append(rvtypes[rvtype])

            # purchase size, sex, height, weight
            if rvbox.select_one('p.review-profile__body_information'):
                meta_pshw.append(rvbox.select_one('p.review-profile__body_information').text.strip())
            else:
                meta_pshw.append('')


            # prd-level-each
            if rvbox.select_one('div.review-evaluation'):
                meta_sizes.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(1)').text.strip())
                meta_brights.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(2)').text.strip())
                meta_colors.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(3)').text.strip())
                meta_thicks.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(4)').text.strip())
            else:
                meta_sizes.append('')
                meta_brights.append('')
                meta_colors.append('')
                meta_thicks.append('')


            rvs.append(rvbox.select_one('div.review-contents__text').text.strip())
            scores.append(rvbox.select_one('span.review-list__rating__active').get('style'))
            helpfuls.append(int(rvbox.select_one('span.review-evaluation-button__count').text))
            uploadAts.append(rvbox.select_one('p.review-profile__date').text.strip())

            collectAts.append(dt.datetime.today())

            i += 1
            ii += 1

        if i == n or ii >= nn:
            break
    ################################ 한 페이지에서 데이터 수집 #

    if i == n or ii >= nn:
        break

    # 현재 페이지가 최종 페이지인지 확인
    if maxp == curr_num:
        print(f'{rvtypes[rvtype]} maxpage도달')
        print()
        if rvtypes[rvtype] == '일반 후기':
            print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
        break
    else: # 최종 페이지가 아니면 다음장 (또는 다음 5페이지 화면)으로 이동
        time.sleep(0.1)
        driver.find_element_by_css_selector(f'div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a:nth-of-type({lp.index(curr_num) + 2})').click()

    time.sleep(0.5)
################################## paging #

current page number : 1, length of rvs : 0


In [ ]:
for cateurl in urls:

    
    # ii : 카테고리별 수집 개수 cnt
    # i : product별 수집 개수 cnt
    ii = 0
    
    # 카테고리로 이동(ex. 반팔티셔)
    driver.get(cateurl)
    time.sleep(3)
    
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    
    #  카테고리(반팔티셔츠)별 제품들의 url들 구하기
    
    driver.get(cateurl)
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
    product_urls = [_['href'] for _ in cate_products]

    for _ in range(4, 13):
        driver.find_element_by_css_selector(f'div.sorter-box.box div.pagination div.wrapper a:nth-of-type({_})').click()
        time.sleep(0.5)
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        page_cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
        page_product_urls = [_['href'] for _ in page_cate_products]
        product_urls += page_product_urls
    #     print(f'_ : {_}, page : {_-2}')

    
    # product 개수 세기
    pn = 0
    
    # product별 url로 이동 #########################################
    for product_url in product_urls:
        pn += 1
        # product로 이동
        driver.get(product_url)
        time.sleep(3)
        
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(0.5)
    
        product_name = dom.select_one('span.product_title em').text.strip()
        category = dom.select_one('p.item_categories a:nth-of-type(2)').text.strip()
        
        
        numofrv = dom.select_one('h3.title-box.font-mss span#review_total').text.strip()
        prdid = product_url.split('/')[-1]
        print(f'상품아이디: {prdid}, 본 상품 총 후기 개수 : {numofrv}')
        # i == n이면 빠져나오기 위함
        # product별 최대 100개씩
        i = 0
        

        # review types 1 : 스타일 후기, 2 : 상품사진후기, 3 : 일반 후기
        # 체험단후기가 있는 경우, 2 : 스타일 후기, 3 : 상품사진후기, 4 : 일반 후기
        if len(dom.select('div#estimateBox div.tabBox > ul.snb > li')) == 4:
            rvtypes = {2 : 'style_estimate_list', 3: 'photo_estimate_list', 4: 'estimate_list'}
        else:
            rvtypes = {1 : 'style_estimate_list', 2: 'photo_estimate_list', 3: 'estimate_list'}
        
        
        
        # review type별 수집 #######################################
        for rvtype in rvtypes:
            
            # review type 화면으로 이동
#             driver.find_element_by_css_selector(f'div#estimateBox div.tabBox > ul.snb > li:nth-of-type({rvtype})').click()
#             time.sleep(1)
    
            # dom 불러오기
#             dom = BeautifulSoup(driver.page_source, 'html.parser')
#             time.sleep(0.5)

            # page description : 최대 몇 페이지까지 있는지 텍스트 ('000페이지 중 000페이지')
            # maxp : 해당 상품의 마지막 페이지 숫자
            if dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg'):
                pagedes = dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg').text.strip()
                maxp = int(re.search(p_maxp, pagedes).group(0).replace(',', ''))
                
                # curr_num : 현재 페이지
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
            else:  # 후기가 없는 경우
                maxp = 0


            print(f'{rvtypes[rvtype]} max page : {maxp}')
    
            # paging ##################################
            while maxp:

                # dom 새로 불러오기
                dom = BeautifulSoup(driver.page_source, 'html.parser')
                time.sleep(0.5)
                
                # 현재 페이지 구하기
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
                print(f'current page number : {curr_num}, length of rvs : {len(rvs)}')
    
                # list of pages : 페이지 리스트
                lp = [int(_.text.strip()) if _.text.strip() != '' else 0 for _ in dom.select('div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a')]
    
                # 한 페이지에서 데이터 수집 ################################
                for rvbox in dom.select(f'div#estimateBox div.tabBox div#{rvtypes[rvtype]} div.review-list'):
                    
                    if rvbox.select_one('span.review-list__rating__active').get('style')[-3] in ['2', '4', '6', '8']:
                    
                        product_names.append(product_name)
                        categories.append(category)

                        retypes.append(rvtypes[rvtype])

                        # purchase size, sex, height, weight
                        if rvbox.select_one('p.review-profile__body_information'):
                            meta_pshw.append(rvbox.select_one('p.review-profile__body_information').text.strip())
                        else:
                            meta_pshw.append('')


                        # prd-level-each
                        if rvbox.select_one('div.review-evaluation'):
                            meta_sizes.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(1)').text.strip())
                            meta_brights.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(2)').text.strip())
                            meta_colors.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(3)').text.strip())
                            meta_thicks.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(4)').text.strip())
                        else:
                            meta_sizes.append('')
                            meta_brights.append('')
                            meta_colors.append('')
                            meta_thicks.append('')


                        rvs.append(rvbox.select_one('div.review-contents__text').text.strip())
                        scores.append(rvbox.select_one('span.review-list__rating__active').get('style'))
                        helpfuls.append(int(rvbox.select_one('span.review-evaluation-button__count').text))
                        uploadAts.append(rvbox.select_one('p.review-profile__date').text.strip())

                        collectAts.append(dt.datetime.today())

                        i += 1
                        ii += 1

                    if i == n or ii >= nn:
                        break
                ################################ 한 페이지에서 데이터 수집 #
                
                if i == n or ii >= nn:
                    break
                
                # 현재 페이지가 최종 페이지인지 확인
                if maxp == curr_num:
                    print(f'{rvtypes[rvtype]} maxpage도달')
                    print()
                    if rvtypes[rvtype] == '일반 후기':
                        print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                    break
                else: # 최종 페이지가 아니면 다음장 (또는 다음 5페이지 화면)으로 이동
                    time.sleep(0.1)
                    driver.find_element_by_css_selector(f'div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a:nth-of-type({lp.index(curr_num) + 2})').click()
    
                time.sleep(0.5)
            ################################## paging #
            
            # 일반 후기까지 다 했거나, i==100이면 리뷰타입 볼 필요 없이 다음 product로 넘어간다
            if i == n or ii >= nn:
                print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                break
        
        ####################################### review type별 수집 #
        
        print(f'product 1개 끝*******************************')
        print()
        
        # 한 카테고리에서 nn개 이상이 쌓이면 다음 카테고리로
        if ii >= nn:
            print(f'category 1개 완료, {category}, 데이터 길이 : {len(product_names)}')
            print()
            break
    ######################################### product별 url로 이동 #
    
    
############################################# category별 url로 이동 #

# 뜯어서 검증 끝

---

# 실제 돌리는 전체코드

In [ ]:
# 수집할 데이터 리스트들 ##############################
product_names = []
categories = []

retypes = []

# purchase size, sex, height, weight
meta_pshw = []
meta_sizes = []
meta_brights = []
meta_colors = []
meta_thicks = []

rvs = []
scores = []
helpfuls = []
uploadAts = []

collectAts = []
############################## 수집할 데이터 리스트들 #

# pattern for max page
p_maxp = re.compile(r'[\d,]+')

n = 100
nn = 5000

# category별 url로 이동 #############################################
for cateurl in urls:
    
    
    # ii : 카테고리별 수집 개수 cnt
    # i : product별 수집 개수 cnt
    ii = 0
    
    # 카테고리로 이동(ex. 반팔티셔)
    driver.get(cateurl)
    time.sleep(3)
    
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    
    #  카테고리(반팔티셔츠)별 제품들의 url들 구하기
    
    driver.get(cateurl)
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
    product_urls = [_['href'] for _ in cate_products]
    
    print(f'카테고리 url : {cateurl}')
    print(f'아이템 url 수집 중...')
    for _ in range(4, 13):
        driver.find_element_by_css_selector(f'div.sorter-box.box div.pagination div.wrapper a:nth-of-type({_})').click()
        time.sleep(0.5)
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        page_cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
        page_product_urls = [_['href'] for _ in page_cate_products]
        product_urls += page_product_urls
    #     print(f'_ : {_}, page : {_-2}')

    
    # product 개수 세기
    pn = 0
    
    # product별 url로 이동 #########################################
    for product_url in product_urls:
        pn += 1
        # product로 이동
        driver.get(product_url)
        time.sleep(3)
        
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(0.5)
    
        product_name = dom.select_one('span.product_title em').text.strip()
        category = dom.select_one('p.item_categories a:nth-of-type(2)').text.strip()
        
        
        numofrv = dom.select_one('h3.title-box.font-mss span#review_total').text.strip()
        prdid = product_url.split('/')[-1]
        print(f'상품아이디: {prdid}, 본 상품 총 후기 개수 : {numofrv}')
        # i == n이면 빠져나오기 위함
        # product별 최대 100개씩
        i = 0
        

        # review types 1 : 스타일 후기, 2 : 상품사진후기, 3 : 일반 후기
        # 체험단후기가 있는 경우, 2 : 스타일 후기, 3 : 상품사진후기, 4 : 일반 후기
        if len(dom.select('div#estimateBox div.tabBox > ul.snb > li')) == 4:
            rvtypes = {2 : 'style_estimate_list', 3: 'photo_estimate_list', 4: 'estimate_list'}
        else:
            rvtypes = {1 : 'style_estimate_list', 2: 'photo_estimate_list', 3: 'estimate_list'}
        
        # review type별 수집 #######################################
        for rvtype in rvtypes:
            
            # review type 화면으로 이동
            driver.find_element_by_css_selector(f'div#estimateBox div.tabBox > ul.snb > li:nth-of-type({rvtype})').click()
            time.sleep(1)
    
            # dom 불러오기
            dom = BeautifulSoup(driver.page_source, 'html.parser')
            time.sleep(0.5)

            # page description : 최대 몇 페이지까지 있는지 텍스트 ('000페이지 중 000페이지')
            # maxp : 해당 상품의 마지막 페이지 숫자
            if dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg'):
                pagedes = dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg').text.strip()
                maxp = int(re.search(p_maxp, pagedes).group(0).replace(',', ''))
                
                # curr_num : 현재 페이지
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
            else:  # 후기가 없는 경우
                maxp = 0


            print(f'{rvtypes[rvtype]} max page : {maxp}')
    
            # paging ##################################
            while maxp:

                # dom 새로 불러오기
                dom = BeautifulSoup(driver.page_source, 'html.parser')
                time.sleep(0.5)
                
                # 현재 페이지 구하기
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
                print(f'current page number : {curr_num}, length of rvs : {len(rvs)}')
    
                # list of pages : 페이지 리스트
                lp = [int(_.text.strip()) if _.text.strip() != '' else 0 for _ in dom.select('div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a')]
    
                # 한 페이지에서 데이터 수집 ################################
                for rvbox in dom.select(f'div#estimateBox div.tabBox div#{rvtypes[rvtype]} div.review-list'):
                    
                    if rvbox.select_one('span.review-list__rating__active').get('style')[-3] not in ['2', '4', '6', '8']:
                    
                        product_names.append(product_name)
                        categories.append(category)

                        retypes.append(rvtypes[rvtype])

                        # purchase size, sex, height, weight
                        if rvbox.select_one('p.review-profile__body_information'):
                            meta_pshw.append(rvbox.select_one('p.review-profile__body_information').text.strip())
                        else:
                            meta_pshw.append('')


                        # prd-level-each
                        if rvbox.select_one('div.review-evaluation'):
                            meta_sizes.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(1)').text.strip())
                            meta_brights.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(2)').text.strip())
                            meta_colors.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(3)').text.strip())
                            meta_thicks.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(4)').text.strip())
                        else:
                            meta_sizes.append('')
                            meta_brights.append('')
                            meta_colors.append('')
                            meta_thicks.append('')


                        rvs.append(rvbox.select_one('div.review-contents__text').text.strip())
                        scores.append(rvbox.select_one('span.review-list__rating__active').get('style'))
                        helpfuls.append(int(rvbox.select_one('span.review-evaluation-button__count').text))
                        uploadAts.append(rvbox.select_one('p.review-profile__date').text.strip())

                        collectAts.append(dt.datetime.today())

                        i += 1
                        ii += 1

                    if i == n or ii >= nn:
                        break
                ################################ 한 페이지에서 데이터 수집 #
                
                if i == n or ii >= nn:
                    break
                
                # 현재 페이지가 최종 페이지인지 확인
                if maxp == curr_num:
                    print(f'{rvtypes[rvtype]} maxpage도달')
                    print()
                    if rvtypes[rvtype] == '일반 후기':
                        print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                    break
                else: # 최종 페이지가 아니면 다음장 (또는 다음 5페이지 화면)으로 이동
                    time.sleep(0.1)
                    driver.find_element_by_css_selector(f'div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a:nth-of-type({lp.index(curr_num) + 2})').click()
    
                time.sleep(0.5)
            ################################## paging #
            
            # 일반 후기까지 다 했거나, i==100이면 리뷰타입 볼 필요 없이 다음 product로 넘어간다
            if i == n or ii >= nn:
                print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                break
        
        ####################################### review type별 수집 #
        
        print(f'product 1개 끝*******************************')
        print()
        
        # 한 카테고리에서 nn개 이상이 쌓이면 다음 카테고리로
        if ii >= nn:
            print(f'category 1개 완료, {category}, 데이터 길이 : {len(product_names)}')
            print()
            break
    ######################################### product별 url로 이동 #
    
    
############################################# category별 url로 이동 #

In [ ]:
df = pd.DataFrame(
    {
     'product_names' :product_names,
     
     'categories' :categories,
     'retypes' :retypes,
     'rvs' :rvs,

     'meta_pshw': meta_pshw,
     'meta_sizes': meta_sizes,
     'meta_brights': meta_brights,
     'meta_colors': meta_colors,
     'meta_thicks': meta_thicks,
     
     'scores' :scores,
     'helpfuls' :helpfuls,
     'uploadAts' :uploadAts,
     'collectAts' :collectAts
     })

In [ ]:
df.to_csv('crawling_20211110_positive.csv')

In [ ]:
df

In [ ]:
df['categories'].value_counts()

셔츠/블라우스       5000
숏 팬츠          5000
데님 팬츠         5000
반팔 티셔츠        5000
트레이닝/조거 팬츠    5000
슈트 팬츠/슬랙스     5000
Name: categories, dtype: int64

In [ ]:
df['retypes'].value_counts()

style_estimate_list    9844
photo_estimate_list    5035
estimate_list          4321
Name: retypes, dtype: int64

In [ ]:
a = []
for p, m in zip(df['product_names'], df['meta_pshw']):
    if p == '주니어 스우시 반바지 BV6865-010':
        if m[0].isalpha():
            a.append('주니어 스우시 반바지 BV6865-010_1512387')
        else:
            a.append('주니어 스우시 반바지 BV6865-010_1293460')
    else:
        a.append(p)
df['product_names'] = a

In [ ]:
a = df['product_names'].value_counts()
a

우먼즈 라운지 스웨트 팬츠 [블랙]                            100
BASIC HALF PANTS (RED) [LRPMCPH701MRDA]        100
우먼즈 슬림 스트레이트 히든 밴딩 슬랙스 [라이트 브라운]               100
우먼즈 웨이스트밴들리스 와이드 슬랙스 [블랙]                      100
사인로고 반팔 티셔츠 블랙                                 100
                                              ... 
Vegan Leather Two tuck Short pants               1
#signature [SHORTS.FIT] Carre jeans.622.pdf      1
우먼즈 오버사이즈 실키 셔츠 [블랙]                             1
우먼즈 오버사이즈 실키 셔츠 [화이트]                            1
STRAIGHT WIDE DENIM PANTS LIGHT BLUE             1
Name: product_names, Length: 289, dtype: int64

# 확인

In [ ]:
df = pd.read_csv('crawling_20211110_positive.csv')

FileNotFoundError: [Errno 2] File crawling_20211110_positive.csv does not exist: 'crawling_20211110_positive.csv'

In [ ]:
df

,Unnamed: 0,product_names,categories,retypes,rvs,meta_pshw,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts
0,0,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길어서 후드 밑에 받쳐입어야할 것 같아요\n약간 가성비..?! 두꺼워서 가을에입기 ...,"여성, 161cm, 57kg",사이즈 커요,밝기 보통이에요,색감 보통이에요,두께감 두꺼워요,width: 60%,0,3시간 전,2021-11-04 23:35:35.399930
1,1,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,오버핏으로 레이어드 해서 입는걸 좋아하는데\n다른 코튼 티셔츠들보다 목부분이 튼튼하...,"여성, 158cm, 50kg",사이즈 커요,밝기 보통이에요,색감 선명해요,두께감 두꺼워요,width: 100%,0,1일 전,2021-11-04 23:35:35.401920
2,2,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길이감이 좋아서 어디 받쳐입기 굉장히 좋아요 그리고 적당히 두꺼워서 맨살이 안 비쳐요!,"남성, 178cm, 68kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2일 전,2021-11-04 23:35:35.402919
3,3,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길이감 좋고 두께도 적당해서 레이어드 하기 좋아요!,"남성, 178cm, 68kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2일 전,2021-11-04 23:35:35.404927
4,4,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,사이즈잘맞아요 핏도좋습니다 감사합니다 잘입을게요,"남성, 175cm, 65kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,0,3일 전,2021-11-04 23:35:35.405932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29995,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,예전에 M사이즈 잘못구매해서 사놓구 교환하는거 잊고있다가 이만한 색상 핏 길이 등등...,NaN,사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,1,2020.08.06,2021-11-05 05:24:11.772895
29996,29996,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,두께 있으면서 얇아서 시원해요\n뭔말인가 싶지만ㅋㅋㅋ 얇아서 주름이 좀 잘 생기고 ...,NaN,NaN,NaN,NaN,NaN,width: 100%,1,2019.08.24,2021-11-05 05:24:11.772895
29997,29997,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,가격도 싸고 쿠폰기간얼마안남아서 쿠폰쓸겨 겸사겸사사봤아여..\n색이 겁나연청이구여 ...,NaN,NaN,NaN,NaN,NaN,width: 100%,4,2018.07.01,2021-11-05 05:24:11.773894
29998,29998,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,예전에 M사이즈 잘못구매해서 사놓구 교환하는거 잊고있다가 이만한 색상 핏 길이 등등...,NaN,사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,1,2020.08.06,2021-11-05 05:24:13.377814


In [ ]:
from collections import Counter

In [ ]:
cntr = Counter(df.rvs)

In [ ]:
sorted(cntr.items(), key=lambda x:x[1], reverse=True)

In [ ]:
df[df.product_names == 'UNBALANCE SHORT PANTS [ICE]']

,Unnamed: 0,product_names,categories,retypes,rvs,meta_pshw,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts
